1. Loading the dataset

In [1]:
# 1.1. Launch commands to automatically reload modules
%load_ext autoreload
%autoreload 2

In [2]:
# 1.2. Import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 1.3. Load the training and test dataset into a dataframes called df_train, df_test and df_target
df_train = pd.read_csv('../data/interim/train_cleaned.csv')
df_test = pd.read_csv('../data/processed/test_cleaned.csv')
df_target = pd.read_csv('../data/interim/target.csv')

In [4]:
df_train.shape

(46618, 93)

In [5]:
df_test.shape

(4970, 93)

In [6]:
df_target.shape

(46618, 1)

2. Spliting the training data into train and validation datasets

In [7]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [9]:
#Split the features and target variable into 2 different sets (training and validation) with 80-20 ratio
X_train, X_val, y_train, y_val = train_test_split(df_train, df_target, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(37294, 93)

In [11]:
X_val.shape

(9324, 93)

In [12]:
y_train.shape

(37294, 1)

In [13]:
y_val.shape

(9324, 1)